 ### Threading / Concurrency Example
 Threads are processes which run in parallel to other threads. In a utopian scenario, if you split a big process in 2 threads, these
 threads will run in parallel so it would take half the time.
 This is not true in most cases. Using CPython, there is a mutex that prevents multiple native threads from executing Python byte
 codes at once. It’s called GIL (global interpreter lock). This lock is necessary mainly because CPython’s memory management
 is not thread-safe, but notice that I/O, image processing, and other potentially blocking operations, happen outside the GIL, so it
 will only become a bottle neck in processes that spend a lot of time inside the GIL.
 In most applications nowadays, concurrency is something we all must be able to handle. Mostly in web applications, where one
 request usually starts a thread, we need to have concurrency and threading in mind so we can write our programs accordingly.
 Threading is also a good solution to optimize response times. Given a scenario in which we have to process 4 million objects, a
 good thing to do would be divide them in 4 groups of a million objects and process them in 4 separated threads.
 
 #### 3.1 Python _thread module
 The _thread moduleis very effective for low level threading, let’s see an example to understand the concept. But keep in mind
 that since Python 2.4, this module is not used anymore.
 The process of spawning a thread is pretty simple. We just need to call a method called start_new_thread, available in
 the _thread module, which receives a function, and arguments to pass to it. It returns immediately and a thread will run in
 parallel. Let’s see:

In [2]:
import _thread as thread
import time

executed_count = 0

# Define a function for the thread
def print_time(thread_name, delay):
    global executed_count
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        print("%s: %s" % (thread_name, time.ctime(time.time())))
    executed_count += 1


# Create two threads as follows
try:
    threads = [
        thread.start_new_thread(print_time, ("Thread-1", 2,)),
        thread.start_new_thread(print_time, ("Thread-2", 4,))
    ]
except:
    print("Error: unable to start thread")

while executed_count < 2:
    pass

Thread-1: Fri Jun 27 21:46:04 2025
Thread-2: Fri Jun 27 21:46:06 2025
Thread-1: Fri Jun 27 21:46:06 2025
Thread-1: Fri Jun 27 21:46:08 2025
Thread-2: Fri Jun 27 21:46:10 2025
Thread-1: Fri Jun 27 21:46:10 2025
Thread-1: Fri Jun 27 21:46:12 2025
Thread-2: Fri Jun 27 21:46:14 2025
Thread-2: Fri Jun 27 21:46:18 2025
Thread-2: Fri Jun 27 21:46:22 2025


So, let's see what is going on:

Then we create two threads, each of them containing our print_time function, with a name and a delay assigned to them. And we have a while which makes sure the program won't exit until executed_count is equal or greater than 2.

#### 3.2 Python threading module

The newer threading module included Python 2.4 provides much more powerful, high-level support for thread than the _thread module.

#### 3.2.1 Extending Thread

The most used procedure for spawning a thread using this module, is defining a subclass of the Thread class. Once you've done it, you should override the __init__and run methods.

Once you've got your class, you just instantiate an object of it and call the method called start. Let's see an example below.

In [7]:
import threading

class MyThread(threading.Thread):
    def __init__(self, name, sleep_time):
        threading.Thread.__init__(self)
        self.name = name
        self.sleep_time = sleep_time

    def run(self):
        print("{} start".format(self.name))
        time.sleep(self.sleep_time)
        print("{} end".format(self.name))


threads = [MyThread("Thread-{}".format(i), i) for i in range(1, 4)]
for t in threads:
    t.start()
    

Thread-1 start
Thread-2 start
Thread-3 start
Thread-1 end
Thread-2 end
Thread-3 end


Of course, we don't need to name our threads like that. Each Thread instance has a name with a default value that can be changed as the thread is created. Naming threads is useful in server processes with multiple service threads handling different operations. 

In [15]:
# def __init__(self, sleep_time):
#     threading.Thread.__init__(self)
#     threading.Thread.__init__(self)
#     self.sleep_time = sleep_time


def __init__(self,sleep_time):
    threading.Thread.__init__(self)
    threading.Thread.__init__(self)
    self.sleep_time=sleep_time

In [17]:
def my_logic(sleep_time):
    thread_name = threading.current_thread().getName()
    print("{} start".format(thread_name))
    time.sleep(sleep_time)
    print("{} end".format(thread_name))


class MyThread(threading.Thread):
    def __init__(self, sleep_time):
        threading.Thread.__init__(self)
        threading.Thread.__init__(self)
        self.sleep_time = sleep_time

    def run(self):
        my_logic(self.sleep_time)


threads = [MyThread(i) for i in range(1, 4)]
for t in threads:
    t.start()

Thread-22 start
Thread-24 start
Thread-26 start


C:\Users\user\AppData\Local\Temp\ipykernel_8200\2092067811.py:2: DeprecationWarning: getName() is deprecated, get the name attribute instead
  thread_name = threading.current_thread().getName()


Thread-22 end
Thread-24 end
Thread-26 end


By executing threading_current_thread(), we gain access to the current thread information. Among that information we can find its status(is_alive()), its, daemon flag (is_daemon()), and other useful methods.

#### 3.2.3 Daemon Threads

Now, let's tals about daemon threads. Until now, our programs waited for every thread to end before actually terminating, but sometimes we don't want that behavior. If we have a thread, pushing status or metrics to a series, we usually don't care if it has finished or not when we shut down our program, and maybe we don't want to explicitly terminate it before exiting.

Daemon threads run without blocking the main thread from exiting. They are useful when we have services where there may not be an easy way to interrupt the thread or where letting the thread die in the middle of its work does not lose or corrupt data.

To spawn a daemon thread, we just spawn a normal thread a call setDaemon() method with True as a parameter. By default thread are not daemon. Let's see how our program behaves when we make those threads daemon:

In [21]:
threads = [MyThread(i) for i in range(1, 4)]
threads[2].setDaemon(True)
for t in threads:
    t.start()

Thread-46 start
Thread-48 start
Thread-50 start


C:\Users\user\AppData\Local\Temp\ipykernel_8200\1277778088.py:2: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  threads[2].setDaemon(True)
C:\Users\user\AppData\Local\Temp\ipykernel_8200\2092067811.py:2: DeprecationWarning: getName() is deprecated, get the name attribute instead
  thread_name = threading.current_thread().getName()


Thread-46 end
Thread-48 end
Thread-50 end


As you can see the main thread is not waiting for Thread-6 to finish before exiting. daemon threads are terminated when the main thread finished its execution.
Let's write something that resembles a real-life problem solution. Make a script, that given an array of URL's, crawls them and saves the htmml in files

In [27]:
# site_crawl.py
import http.client
import threading
import logging

logging.basicConfig(level=logging.INFO, format='(%(threadName)-10s) %(message)s', )

def save(html, file_absolute_path):
    logging.info("saving {} bytes to {}".format(len(html), file_absolute_path))
    with open(file_absolute_path, 'wb+') as file:
        file.write(html)
        file.flush()


def crawl(req):
    logging.info("executing get request for parameters: {}".format(str(req)))
    connection = http.client.HTTPConnection(req["host"], req["port"])
    connection.request("GET", req['path'])
    response = connection.getresponse()
    logging.info("got {} response http code".format(response.status))
    logging.debug("headers: {}".format(str(response.headers)))
    response_content = response.read()
    logging.debug("actual response: {}".format(response_content))
    return response_content


class MyCrawler(threading.Thread):
    def __init__(self, req, file_path):
        threading.Thread.__init__(self, name="Crawler-{}".format(req["host"]))
        self.req = req
        self.file_path = file_path


    def run(self):
        global executed_crawlers
        html = crawl(self.req)
        save(html, self.file_path)


def __main__():
    continue_input = True
    threads = []
    while continue_input:
        host = input("host: ")
        port = 80 # int(input("port: "))
        path = "/" # input("path: ")
        file_path = input("output file absolute path: ")
        req = {"host": host, "port": port, "path": path}
        threads.append(MyCrawler(req, file_path))
        continue_input = input("add another? (y/N) ") == "y"

    for t in threads:
        t.start()

__main__()

host:  www.google.com
output file absolute path:  /tmp/google-home.html
add another? (y/N)  y
host:  www.bbc.com
output file absolute path:  /tmp/bbc-home.html
add another? (y/N)  N


(Crawler-www.google.com) executing get request for parameters: {'host': 'www.google.com', 'port': 80, 'path': '/'}
(Crawler-www.bbc.com) executing get request for parameters: {'host': 'www.bbc.com', 'port': 80, 'path': '/'}
Exception in thread Crawler-www.google.com:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Crawler-www.bbc.com:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\user\AppData\Local\Temp\ipykernel_8200\792128257.py", line 36, in run
    self.run()
  File "C:\Users\user\AppData\Local\Temp\ipykernel_8200\792128257.py", line 36, in run
  File "C:\Users\user\AppData\Local\Temp\ipykernel_8200\792128257.py", line 18, in crawl
  File "C:\Users\user\AppData\Local\Temp\ipykernel_8200\792128257.py", line 18, in crawl
  File "C:\Users\user\anaconda3\Lib\http\client.py", line 1336, in request


#### 3.2.5

Another thing that's worthy of being pointed out, is thee existence of the class threading.Timer. It is basically a subclass of Thread which, given a delay and function, it executes the function after delay has passed. Also, it can be cancelled at any point.

In [11]:
import threading
import time
import logging


logging.basicConfig(level=logging.DEBUG, format='(%(threadName)-10s) %(message)s',)

def delayed():
    logging.debug('worker running')
    return

t1 = threading.Timer(3, delayed)
t1.name = 't1'
t2 = threading.Timer(3, delayed)
t2.name = 't2'

logging.debug('starting timers')
t1.start()
t2.start()

logging.debug('waiting before canceling %s', t2.name)
time.sleep(2)
logging.debug('canceling %s', t1.name)
t2.cancel()
logging.debug('done')

(MainThread) starting timers
(MainThread) waiting before canceling t2
(MainThread) canceling t1
(MainThread) done
(t1        ) worker running


Here, we are creating two timers, both execute the same function after 3 seconds. Then we wait 2 seconds and cancel one of them. In the output we can see only one of the timers executed the delayed function.
This is useful on scenarios where we need to execute some process if something didn't happen in an interval of time, or even for schedule.

#### 3.2.6 Events: Communication Between Threads

New, we all know that idea of using threds is making tasks independent from each other, but some times we need for a thread to wait for an event caused by another. Python provides a way of signaling between threads. To exxperiment with this, we'll make a race


In [16]:
#race.py
import threading

class Racer(threading.Thread):

    def __init__(self, name, start_signal):
        threading.Thread.__init__(self, name=name)
        self.start_signal = start_signal

    def run(self):
        self.start_signal.wait()
        print("I, {}, got to the goal!".format(self.name))

# class Race:

#     def __init__(self, racer_names):
#         self.start_signal = threading.Event()
#         self.racers = [Racer(name, self.start_signal) for name in racer_names]
#         for racer in self.racers:
#             racer.start()

#     def start(self):
#         self.start_signal.set()

def __main__():
    race = Race(["rabbit", "turtle", "cheetah", "monkey", "cow", "horse", "tiger", "lion"])
    race.start()


__main__()

I, rabbit, got to the goal!I, turtle, got to the goal!
I, cheetah, got to the goal!
I, monkey, got to the goal!
I, cow, got to the goal!
I, horse, got to the goal!
I, tiger, got to the goal!
I, lion, got to the goal!

